In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)



%cd /content/drive/MyDrive/projects_AI/SmartWiki


Mounted at /content/drive
/content/drive/MyDrive/projects_AI/SmartWiki


In [2]:
import pandas as pd

df = pd.read_excel("data/wiki_excel.xlsx")
url = df["url"].tolist()
print(url[0])

https://en.wikipedia.org/wiki/Cognizant


In [3]:
import requests
r = requests.get(url[0], headers = {"User-Agent": "Mozilla/5.0"})
html = r.text


In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, "lxml")
print(soup)

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Cognizant - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-

In [16]:
from bs4 import BeautifulSoup
import re
soup = BeautifulSoup(html, "lxml")

# Find the div with the id "mw-content-text" which contains the main content
content_div = soup.find("div", {"id": "mw-content-text"})

# Extract text from the content div if found, otherwise use the full text as a fallback
if content_div:
    paragraphs = content_div.find_all("p")
else:
    text = soup.get_text() # Fallback to full text if content div is not found

clean_text= []
for p in paragraphs:
  text = p.get_text()
  text = re.sub(r"\[\d+\]"," ",text)
  clean_text.append(text.strip())
article_text = "\n".join(clean_text)

# Print the first 500 characters of the extracted text
print(article_text[:3000])

Cognizant Technology Solutions Corporation is an American multinational information technology consulting and outsourcing company originally founded in India.  It is headquartered in Teaneck, New Jersey, United States. Cognizant is part of the NASDAQ-100 and trades under CTSH.  It was founded in Chennai, India, as an in-house technology unit of Dun & Bradstreet in 1994,  and started serving external clients in 1996.  After a series of corporate reorganizations, there was an initial public offering in 1998.  Ravi Kumar Singisetti has been the CEO of the company since January 2023, replacing Brian Humphries.
Cognizant was established in 1994 in Chennai, India, as Dun & Bradstreet Satyam Software (DBSS), a 76:24 joint venture between Dun & Bradstreet and Satyam Computers, with Kumar Mahadeva, and Srini Raju as the founding CEOs and MDs.   It began with 50 employees in Chennai as Dun & Bradstreet's in-house technology unit focused on implementing large-scale IT projects for Dun & Bradstree

In [22]:
title = soup.find("title")
print(title.get_text().replace(" - Wikipedia","")+".txt")

Cognizant.txt


In [17]:
import os

# Define the data folder path
data_folder = "data"

# Create the data folder if it doesn't exist
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

# Define the output file path
output_file = os.path.join(data_folder, "extracted_text.txt")

# Write the extracted text to the file
with open(output_file, "w", encoding="utf-8") as f:
    f.write(text)

print(f"Text successfully saved to {output_file}")

Text successfully saved to data/extracted_text.txt


In [26]:
import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [27]:
with open("data/Cognizant.txt","r", encoding="utf-8") as f:
  text = f.read()

sentences = sent_tokenize(text)
print(sentences[:4])

['Cognizant Technology Solutions Corporation is an American multinational information technology consulting and outsourcing company originally founded in India.', 'It is headquartered in Teaneck, New Jersey, United States.', 'Cognizant is part of the NASDAQ-100 and trades under CTSH.', 'It was founded in Chennai, India, as an in-house technology unit of Dun & Bradstreet in 1994,  and started serving external clients in 1996.']


In [35]:
chunk_size = 300
chunks= []
current_chunk = ""
for s in sentences:
  if len(current_chunk)+ len(s) <=chunk_size:
    current_chunk += s+ " "
  else:
    if current_chunk.strip():
      chunks.append(current_chunk.strip())
      current_chunk = s+ " "
if current_chunk.strip():
  chunks.append(current_chunk.strip())

print(chunks[1])



It was founded in Chennai, India, as an in-house technology unit of Dun & Bradstreet in 1994,  and started serving external clients in 1996. After a series of corporate reorganizations, there was an initial public offering in 1998.


In [38]:
import json
def load_chunk():
  with open("artifacts/chunks/final_chunks.json", "r", encoding="utf-8") as file:
    data = json.load(file)
  return data

chunks = load_chunk()
print(chunks[0])

{'doc_id': 'Cognizant', 'chunk_id': 0, 'text': 'Cognizant Technology Solutions Corporation is an American multinational information technology consulting and outsourcing company originally founded in India. It is headquartered in Teaneck, New Jersey, United States. Cognizant is part of the NASDAQ-100 and trades under CTSH. It was founded in Chennai, India, as an in-house technology unit of Dun & Bradstreet in 1994,  and started serving external clients in 1996.'}
